# Linear Discriminant Analysis (LDA)
- The covariance matrix can tell us about the scatter within a dataset, which is the amount of spread that there is within the data. The way to find this scatter is to multiply
the covariance by the $p_c$ , the probability of the class (that is, the number of datapoints
there are in that class divided by the total number). Adding the values of this for all of the
classes gives us a measure of the **within-class scatter** of the dataset:

\begin{equation}
S_W = \sum_{c \in classes} \sum_{j \in c} p_c(x_j - \mu_c)(x_j - \mu_c)^T .
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data
labels = iris.target
target_ids = np.unique(labels)#labels = array([0, 1, 2])
Ndata = np.shape(data)[0]
Sw = np.zeros((4, 4))
sb = 0
mc = np.mean(data, axis=0)

C = np.cov(np.transpose(data))
# Loop over classes
classes = np.unique(labels)
for i in range(len(classes)):
# Find relevant datapoints
    indices = np.squeeze(np.where(labels==classes[i]))
    d = np.squeeze(data[indices,:])
    classcov = np.cov(d.T)
    Sw += np.float(np.shape(indices)[0])/Ndata * classcov
    m = np.mean(d, axis=0)
    sb += np.dot( (m-mc).reshape(4,1) , (m-mc).reshape(1,4) )/3
    print ('Sw: \n',Sw)

print('sb\n',sb,'\n')
print(C - Sw)

Sw: 
 [[0.04141633 0.03307211 0.0054517  0.00344354]
 [0.03307211 0.0478966  0.00389932 0.00309932]
 [0.0054517  0.00389932 0.01005306 0.00202313]
 [0.00344354 0.00309932 0.00202313 0.00370204]]
Sw: 
 [[0.13022721 0.06146667 0.06641769 0.02203673]
 [0.06146667 0.08071973 0.03145034 0.01683401]
 [0.06641769 0.03145034 0.0836585  0.02639048]
 [0.02203673 0.01683401 0.02639048 0.01673741]]
Sw: 
 [[0.26500816 0.09272109 0.16751429 0.03840136]
 [0.09272109 0.11538776 0.05524354 0.0327102 ]
 [0.16751429 0.05524354 0.18518776 0.04266531]
 [0.03840136 0.0327102  0.04266531 0.04188163]]
sb
 [[ 0.42141422 -0.13301778  1.101656    0.47519556]
 [-0.13301778  0.07563289 -0.38159733 -0.15288444]
 [ 1.101656   -0.38159733  2.91401867  1.24516   ]
 [ 0.47519556 -0.15288444  1.24516     0.53608889]] 

[[ 0.42068535 -0.13515509  1.10680115  0.47786933]
 [-0.13515509  0.07459166 -0.38489991 -0.15434958]
 [ 1.10680115 -0.38489991  2.9310901   1.25294409]
 [ 0.47786933 -0.15434958  1.25294409  0.53912463]]

If our dataset is easy to separate into classes, then this within-class scatter should be
small, so that each class is tightly clustered together. However, to be able to separate the
data, we also want the distance between the classes to be large. This is known as the **between-
classes scatter** and is a significantly simpler computation, simply looking at the difference
in the means:

\begin{equation}
S_B = \sum_{c \in classes} (\mu_c - \mu)(\mu_c - \mu)^T .
\end{equation}


In [ ]:
Sb = C - Sw

The argument about good separation suggests that datasets that are easy to separate into the different classes
should have  $S_B /S_W$ as large as possible.

The projection of
the data can be written as $z = w^T \cdot x$ for datapoint $x$. 

Replacing $x_j$ with $w^T \cdot x_j$, we can use some linear algebra to get:

\begin{equation}
\sum_{c \in classes} \sum_{j \in c}
p_c(w^T \cdot (x_j - \mu_c))(w^{T} \cdot (x_j - \mu_c))^T = w^T S_W w
\end{equation}

\begin{equation}
\sum_{c \in classes}
w^T (\mu_c - \mu)(\mu_c - \mu)^Tw = w^T S_B w .
\end{equation}

So our ratio of within-class and between-class scatter looks like:

\begin{equation}
\frac{w^T S_W w}{w^T S_B w}.
\end{equation}

In order to find the maximum value of this with respect to $w$, we differentiate it and set
the derivative equal to 0. This tells us that:

\begin{equation}
\frac{S_B w(w^TS_W w)-S_W w(w^T S_B w)}{(w^T S_W w)^2}S_B w = 0 .
\end{equation}

So we just need to solve this equation for w and we are done. We start with a little bit
of rearranging to get:

\begin{equation}
S_Ww = \frac{w^T S_W w}{w^T S_B w}S_B w.
\end{equation}

Finding the minimum requires computing the generalised eigenvectors of $S_W^{-1} S_B$

In [ ]:
evals,evecs = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
print np.linalg.inv(Sw).dot(Sb)

In [ ]:
print evecs.dot(np.linalg.inv(Sw).dot(Sb)).dot(evecs.T)

In [ ]:
print evecs.T.dot(Sb).dot(evecs)/evecs.T.dot(Sw).dot(evecs)

In [ ]:
evals

In [ ]:
evals,evecs = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
indices = np.argsort(evals)
indices = indices[::-1]
evecs = evecs[:,indices]
evals = evals[indices]
w = evecs[:,:2]
n = np.dot(data,w)

colors = ['navy', 'turquoise', 'darkorange']

for i, c,label in zip(target_ids, colors, iris.target_names):
    plt.scatter(n[i == labels, 0], n[i == labels, 1], c = c, edgecolors='black', s=235,label=label, marker='.')
plt.legend()
plt.show()

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

X = iris.data
y = iris.target

lda = lda(n_components=2)
X_lda = lda.fit(X, y).transform(X)
for i, c, label in zip(target_ids, colors,iris.target_names):
    plt.scatter(X_lda[i == labels, 0], X_lda[i == labels, 1], c = c, edgecolors='black', s=235,label=label, marker='.')
plt.legend()
plt.show()